<a href="https://colab.research.google.com/github/Dheeraj3reddy/hello-world/blob/master/IITM_RL_Final_Project_Release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IITM RL FINAL PROJECT

## Problem - bsuite 

This notebook uses an open source reinforcement learning benchmark known as bsuite. https://github.com/deepmind/bsuite

bsuite is a collection of carefully-designed experiments that investigate core capabilities of a reinforcement learning agent.

Your task is to use any reinforcement learning techniques at your disposal to get high scores on the environments specified.

**Note**: Since the course is on Reinforcement Learning,  please limit yourself to using traditional Reinforcement Learning algorithms, 

**Do not use deep reinforcement learning.**

# How to use this notebook? 📝

- This is a shared template and any edits you make here will not be saved. **You
should make a copy in your own drive**. Click the "File" menu (top-left), then "Save a Copy in Drive". You will be working in your copy however you like.

<p style="text-align: center"><img src="https://gitlab.aicrowd.com/aicrowd/assets/-/raw/master/notebook/aicrowd_notebook_submission_flow.png?inline=false" alt="notebook overview" style="width: 650px;"/></p>

- **Update the config parameters**. You can define the common variables here

Variable | Description
--- | ---
`AICROWD_RESULTS_DIR` | Path to write the output to.
`AICROWD_ASSETS_DIR` | In case your notebook needs additional files (like model weights, etc.,), you can add them to a directory and specify the path to the directory here (please specify relative path). The contents of this directory will be sent to AIcrowd for evaluation.
`AICROWD_API_KEY` | In order to submit your code to AIcrowd, you need to provide your account's API key. This key is available at https://www.aicrowd.com/participants/me

- **Installing packages**. Please use the [Install packages 🗃](#install-packages-) section to install the packages

In [ ]:
!pip install -q aicrowd-cli

     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 204kB 7.0MB/s 
     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 163kB 9.3MB/s 
     |████████████████████████████████| 61kB 4.4MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 71kB 5.4MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


# AIcrowd Runtime Configuration 🧷

Get login API key from https://www.aicrowd.com/participants/me


In [ ]:
import os

AICROWD_RESULTS_DIR = os.getenv("OUTPUTS_DIR", "results")
os.environ["RESULTS_DIR"] = AICROWD_RESULTS_DIR
API_KEY = "c486fada045b25f684460d5da87749d1"

In [ ]:
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


# Install packages 🗃

Please add all pacakage installations in this section

In [ ]:
!pip install git+http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git
!pip install tabulate
!pip install tqdm

## Add any other installations you need here

  Cloning http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git to /tmp/pip-req-build-q8i5o3el
  Running command git clone -q http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git /tmp/pip-req-build-q8i5o3el
  Created wheel for bsuite: filename=bsuite-0.3.5-cp37-none-any.whl size=252043 sha256=7afbcb796811b3ebc48ec75fcc2e4286f95d8545f19d1086b36655581704b613
  Stored in directory: /tmp/pip-ephem-wheel-cache-63k458yn/wheels/61/ea/06/77c82c07765fb8608e50e6c66bc566fa6d113c725bc6937e7b
Successfully built bsuite


# Import packages

In [ ]:
import gym
import warnings

import numpy as np
import pandas as pd
import plotnine as gg
from tqdm.notebook import tqdm

import bsuite
from bsuite.aicrowd import environments
from bsuite.aicrowd.runner import Runner
from bsuite.aicrowd.analysis import Analyzer

pd.options.mode.chained_assignment = None
gg.theme_set(gg.theme_bw(base_size=16, base_family='serif'))
gg.theme_update(figure_size=(3, 1), panel_spacing_x=0.5, panel_spacing_y=0.5)
warnings.filterwarnings('ignore')

# **Agent Class**

You can modify the AGENT TEMPLATE below and implement the logic of your agent. Your agent must implement a few methods that will be called by the `Runner` class.
* `__init__` - put any initialization code here.
* `get_action` - takes in a `state` and returns an `action`.
* `learn` - takes in `(state, action, reward, next_state)`, implements the learning logic.
* `get_state` - takes in a raw `observation` directly from the env, discretizes it and returns a `state`.

In addition to these, you may implement other methods which can be called by the above methods.

Since there are multiple environments, you may need unique hyper parameters for each environment. Instantiate the agent while passing in the hyper parameters in a dictionary using the `agent_config` parameter so that each environment can use different hyper parameters for the agent while using a single `Agent` class for all of them.  You can use any names for the keys in the config dictionary.   

An example `RandomAgent` is given below.

In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# AGENT TEMPLATE
class Agent:
    def __init__(self, agent_config):
        self.config = agent_config
        self.env_name = self.config['env_name']
        self.n_states=self.config['n_states']
        self.iter_max=self.config['iter_max']
        self.initial_lr=self.config['initial_lr']
        self.min_lr=self.config['min_lr']
        self.gamma=self.config['gamma']
        self.t_max=self.config['t_max']
        self.eps=self.config['eps']
        self.q_table = self.config['q_table']
        self.env=self.config['env']
        self.env1=gym.make('MountainCar-v0')
        self.i=0
        self.j=0
        pass

    def get_action(self, state):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
        RETURNS     : 
            - action - 'action' to be taken
        '''
        
        if np.random.uniform(0, 1) < self.eps:
            return np.random.choice(self.env.action_space.n)
        else:
            logits = self.q_table[state[0]][state[1]]
            logits_exp = np.exp(logits)
            probs = logits_exp / np.sum(logits_exp)
            action = np.random.choice(self.env.action_space.n, p=probs)
        return action
    
    def learn(self, state, action, reward, next_state):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
            - action - 'action' performed in 'state'
            - reward - 'reward' received due to action taken
            - next_state - discretized 'next_state'
        RETURNS     : 
            - NIL
        '''
        
       
        #print(eta,i,j)
        self.q_table[state[0]][state[1]][action] = self.q_table[state[0]][state[1]][action] + eta * (reward + self.gamma *  np.max(self.q_table[next_state[0]][next_state[1]]) - self.q_table[state[0]][state[1]][action])

    def get_state(self, obs):
        '''
        PARAMETERS  : 
            - observation - raw 'observation' from environment
        RETURNS     : 
            - state - discretized 'state' from raw 'observation'
        '''
        obs=obs.T
        env_low = self.env1.observation_space.low
        env_high = self.env1.observation_space.high
        env_dx = (env_high - env_low) / n_states
        a = int((obs[0] - env_low[0])/env_dx[0])
        b = int((obs[1] - env_low[1])/env_dx[1])
        state= [ a,b,obs[2]]

        return state



In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# DO NOT rename the config dictionaries as the evaluator references them. However, you may use any names for the keys in them.
catch_config = {"env_name": "catch"}
catch_noise_config = {"env_name": "catch_noise"}
cartpole_config = {"env_name": "cartpole"}
cartpole_noise_config = {"env_name": "cartpole_noise"}
mountaincar_config = {"env_name": "mountaincar"}
mountaincar_noise_config = {"env_name": "mountaincar_noise"}

In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# EXAMPLE
class RandomAgent:
    def __init__(self, agent_config={}):
        self.config = agent_config
        self.env_name = self.config['env_name']

    def get_action(self, state):
        action = np.random.choice(2)
        return action
    
    def learn(self, state, action, reward, next_state):
        if ('BAR' in self.config):
            if (self.config['BAR']):
                self.config['FOO'] += 1

    def get_state(self, observation):
        # In this function you're allowed to use 
        # the environment name for observation preprocessing
        # Do not use it anywhere else
        if self.env_name == 'catch':
          state = observation
        elif self.env_name == 'catch_noise':
          state = observation
        elif self.env_name == 'cartpole':
          state = observation
        elif self.env_name == 'cartpole_noise':
          state = observation
        elif self.env_name == 'mountaincar':
          state = observation
        elif self.env_name == 'mountaincar_noise':
          state = observation
        else:
          raise NotImplementedError

        return state

env1_config = {
    "env_name": 'cartpole',
    'FOO': 0.1,
    'BAR': True
}

env2_config = {
    "env_name": 'cartpole',
    'FOO': 0.2,
    'BAR': False
}

randomAgent1 = RandomAgent(agent_config=env1_config)
randomAgent2 = RandomAgent(agent_config=env2_config)

# **Playing with the Environment**

#### **Instantiating the environment** :
You can create an environment by calling the following function:  
`environments.load_env(ENV_ID)` - RETURNS: `env`  
where, ENV_ID can be ONE of the following:
* `environments.CATCH`
* `environments.CATCH_NOISE`
* `environments.CARTPOLE`
* `environments.CARTPOLE_NOISE`
* `environments.MOUNTAINCAR`
* `environments.MOUNTAINCAR_NOISE`

The `NOISE` environments add a scaled random noise to the `reward`.
<br/>

#### **Runnning the environment** :
There are certain methods required to run the environments. The interface is very similar to OpenAI Gym's interfaces. Fore more information, read the OpenAI documentation [here](https://gym.openai.com/docs/).

`env.reset()` - RETURNS: `observation`  
`env.step(action)`  - RETURNS: `(next_observation, reward, done, info[NOT USED])`

There are also a few useful properties within the environments:

* `env.action_space.n` - total number of possible actions. eg: if 'n' is 3, then the possible actions are `[0, 1, 2]`
* `env.observation_space.shape` -  the shape of the observation.
* `env.bsuite_num_episodes` -  the pre-specified number of episodes which will be run during evaluation (unique for each environment).

##### *ONLY IN CATCH / CATCH_NOISE*
* `env.observation_space.high` -  the upper limit for every index in the observation.
* `env.observation_space.low` -  the lower limit for every index of the observation.
<br/>


## **Environment Observation Space Limits:**

The limits for the observation space (minimum and maximum) for all the environments are given in the table below:

| Environments                        | Limits                                                                      |
|-------------------------------------|-----------------------------------------------------------------------------|
| CATCH <br/>  CATCH_NOISE            | MIN: use `env.observation_space.low` <br/> MAX: use `env.observation_space.high` |
| CARTPOLE <br/> CARTPOLE_NOISE       | MIN: `[-1. -5., -1., -1., -5., 0.]` <br/> MAX: `[ 1.,  5.,  1.,  1.,  5., 1.]` |
| MOUNTAINCAR <br/> MOUNTAINCAR_NOISE | MIN: `[-1.2, -0.07, 0.]` <br/> MAX: `[ 0.6,  0.07,  1.]`                                 |

[NOTE] Use this code cell to play around and get used to the environments. However, the `Runner` class below will be used to evaluate your agent.

In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# TEST AREA
env = environments.load_env(environments.CARTPOLE)  # replace 'environments.CARTPOLE' with other environments

agent = RandomAgent(agent_config={"env_name": 'cartpole'})    # replace with 'RandomAgent()' to use your custom agent

NUM_EPISODES = 10                                   # replace with 'env.bsuite_num_episodes' to run for pre-specified number of episodes
for episode_n in tqdm(range(NUM_EPISODES)):
    done = False
    episode_reward = 0
    episode_moves = 0
 
    observation = env.reset()
    state = agent.get_state(observation)

    while not done:
        action = agent.get_action(state)

        next_observation, reward, done, _ = env.step(action)
        next_state = agent.get_state(next_observation)

        agent.learn(state, action, reward, next_state)

        state = next_state

        episode_reward += reward
        episode_moves += 1

    if (((episode_n+1) % 2) == 0): 
        print("EPISODE: ",episode_n+1,"\tREWARD: ",episode_reward,"\tEPISODE_LENGTH: ",episode_moves)

Loaded bsuite_id: cartpole/0.


  0%|          | 0/10 [00:00<?, ?it/s]

EPISODE:  2 	REWARD:  39.0 	EPISODE_LENGTH:  40
EPISODE:  4 	REWARD:  51.0 	EPISODE_LENGTH:  52
EPISODE:  6 	REWARD:  44.0 	EPISODE_LENGTH:  45
EPISODE:  8 	REWARD:  39.0 	EPISODE_LENGTH:  40
EPISODE:  10 	REWARD:  37.0 	EPISODE_LENGTH:  38


## Point to the Agent Class you'll use for the final score

In [ ]:
RLAgent = RandomAgent

# **Evaluating the Agent on all the Environments**

* The following cells will take care of running your agent on each environment and aggregating the results in csv files. In each of the following cells, the `agent_config` parameter is already set to use the corresponding config dictionary for that environment. DO NOT EDIT THIS.
* Feel free to modify the `LOG_INTERVAL` parameter to change the interval between episodes for logging.  
* Please do not modify any other contents in each of the cells.  

In [ ]:
LOG_INTERVAL = 100

In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=catch_config),
    env_id = environments.CATCH,
    log_interval = LOG_INTERVAL,
)
runner.play_episodes()

Loaded bsuite_id: catch/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/10000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -1.0 	MEAN_REWARD:  -0.52 	EPISODE_LENGTH:  9
EPISODE:  200 	REWARD:  -1.0 	MEAN_REWARD:  -0.46 	EPISODE_LENGTH:  9
EPISODE:  300 	REWARD:  -1.0 	MEAN_REWARD:  -0.6 	EPISODE_LENGTH:  9
EPISODE:  400 	REWARD:  -1.0 	MEAN_REWARD:  -0.62 	EPISODE_LENGTH:  9
EPISODE:  500 	REWARD:  -1.0 	MEAN_REWARD:  -0.7 	EPISODE_LENGTH:  9
EPISODE:  600 	REWARD:  -1.0 	MEAN_REWARD:  -0.64 	EPISODE_LENGTH:  9
EPISODE:  700 	REWARD:  -1.0 	MEAN_REWARD:  -0.74 	EPISODE_LENGTH:  9
EPISODE:  800 	REWARD:  -1.0 	MEAN_REWARD:  -0.46 	EPISODE_LENGTH:  9
EPISODE:  900 	REWARD:  -1.0 	MEAN_REWARD:  -0.68 	EPISODE_LENGTH:  9
EPISODE:  1000 	REWARD:  -1.0 	MEAN_REWARD:  -0.58 	EPISODE_LENGTH:  9
EPISODE:  1100 	REWARD:  -1.0 	MEAN_REWARD:  -0.56 	EPISODE_LENGTH:  9
EPISODE:  1200 	REWARD:  -1.0 	MEAN_REWARD:  -0.6 	EPISODE_LENGTH:  9
EPISODE:  1300 	REWARD:  -1.0 	MEAN_REWARD:  -0.58 	EPISODE_LENGTH:  9
EPISODE:  1400 	REWARD:  1.0 	MEAN_REWARD:  -0.5 	EPISODE_LENGTH:  9
EPISODE:  1500 	REWA

In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=catch_noise_config),
    env_id = environments.CATCH_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: catch_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/10000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -2.4319180485080194 	MEAN_REWARD:  -0.53 	EPISODE_LENGTH:  9
EPISODE:  200 	REWARD:  0.8881949106608489 	MEAN_REWARD:  -0.46 	EPISODE_LENGTH:  9
EPISODE:  300 	REWARD:  -1.2941227075745902 	MEAN_REWARD:  -0.63 	EPISODE_LENGTH:  9
EPISODE:  400 	REWARD:  -1.4250488015594018 	MEAN_REWARD:  -0.62 	EPISODE_LENGTH:  9
EPISODE:  500 	REWARD:  -0.18346899641600334 	MEAN_REWARD:  -0.59 	EPISODE_LENGTH:  9
EPISODE:  600 	REWARD:  0.39936420173707576 	MEAN_REWARD:  -0.39 	EPISODE_LENGTH:  9
EPISODE:  700 	REWARD:  -0.6115223148558895 	MEAN_REWARD:  -0.71 	EPISODE_LENGTH:  9
EPISODE:  800 	REWARD:  1.8585504709523746 	MEAN_REWARD:  -0.68 	EPISODE_LENGTH:  9
EPISODE:  900 	REWARD:  -0.637721569542689 	MEAN_REWARD:  -0.75 	EPISODE_LENGTH:  9
EPISODE:  1000 	REWARD:  -0.1701424555746447 	MEAN_REWARD:  -0.61 	EPISODE_LENGTH:  9
EPISODE:  1100 	REWARD:  -2.373967525284758 	MEAN_REWARD:  -0.53 	EPISODE_LENGTH:  9
EPISODE:  1200 	REWARD:  0.9410664532715053 	MEAN_REWARD:  -0.64 	

In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=cartpole_config),
    env_id = environments.CARTPOLE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: cartpole/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  48.0 	MEAN_REWARD:  39.62 	EPISODE_LENGTH:  49
EPISODE:  200 	REWARD:  38.0 	MEAN_REWARD:  39.25 	EPISODE_LENGTH:  39
EPISODE:  300 	REWARD:  46.0 	MEAN_REWARD:  39.25 	EPISODE_LENGTH:  47
EPISODE:  400 	REWARD:  41.0 	MEAN_REWARD:  39.33 	EPISODE_LENGTH:  42
EPISODE:  500 	REWARD:  39.0 	MEAN_REWARD:  39.22 	EPISODE_LENGTH:  40
EPISODE:  600 	REWARD:  41.0 	MEAN_REWARD:  39.25 	EPISODE_LENGTH:  42
EPISODE:  700 	REWARD:  37.0 	MEAN_REWARD:  38.76 	EPISODE_LENGTH:  38
EPISODE:  800 	REWARD:  44.0 	MEAN_REWARD:  39.38 	EPISODE_LENGTH:  45
EPISODE:  900 	REWARD:  32.0 	MEAN_REWARD:  39.29 	EPISODE_LENGTH:  33
EPISODE:  1000 	REWARD:  34.0 	MEAN_REWARD:  39.98 	EPISODE_LENGTH:  35


In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=cartpole_noise_config),
    env_id = environments.CARTPOLE_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: cartpole_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  38.695912728165425 	MEAN_REWARD:  39.19 	EPISODE_LENGTH:  39
EPISODE:  200 	REWARD:  42.88700013836746 	MEAN_REWARD:  39.67 	EPISODE_LENGTH:  44
EPISODE:  300 	REWARD:  34.44267331882654 	MEAN_REWARD:  39.09 	EPISODE_LENGTH:  37
EPISODE:  400 	REWARD:  40.758980865553106 	MEAN_REWARD:  39.6 	EPISODE_LENGTH:  43
EPISODE:  500 	REWARD:  45.03786307311194 	MEAN_REWARD:  38.47 	EPISODE_LENGTH:  43
EPISODE:  600 	REWARD:  41.87206267978895 	MEAN_REWARD:  38.93 	EPISODE_LENGTH:  42
EPISODE:  700 	REWARD:  40.00846320658314 	MEAN_REWARD:  38.6 	EPISODE_LENGTH:  42
EPISODE:  800 	REWARD:  38.303728191614994 	MEAN_REWARD:  39.41 	EPISODE_LENGTH:  40
EPISODE:  900 	REWARD:  38.94137451395413 	MEAN_REWARD:  39.36 	EPISODE_LENGTH:  38
EPISODE:  1000 	REWARD:  37.05482930725222 	MEAN_REWARD:  39.38 	EPISODE_LENGTH:  37


In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=mountaincar_config),
    env_id = environments.MOUNTAINCAR,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: mountain_car/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -1000.0 	MEAN_REWARD:  -1000.0 	EPISODE_LENGTH:  1000
EPISODE:  200 	REWARD:  -1000.0 	MEAN_REWARD:  -1000.0 	EPISODE_LENGTH:  1000
EPISODE:  300 	REWARD:  -1000.0 	MEAN_REWARD:  -1000.0 	EPISODE_LENGTH:  1000
EPISODE:  400 	REWARD:  -1000.0 	MEAN_REWARD:  -1000.0 	EPISODE_LENGTH:  1000
EPISODE:  500 	REWARD:  -1000.0 	MEAN_REWARD:  -1000.0 	EPISODE_LENGTH:  1000
EPISODE:  600 	REWARD:  -1000.0 	MEAN_REWARD:  -1000.0 	EPISODE_LENGTH:  1000
EPISODE:  700 	REWARD:  -1000.0 	MEAN_REWARD:  -1000.0 	EPISODE_LENGTH:  1000
EPISODE:  800 	REWARD:  -1000.0 	MEAN_REWARD:  -1000.0 	EPISODE_LENGTH:  1000
EPISODE:  900 	REWARD:  -1000.0 	MEAN_REWARD:  -1000.0 	EPISODE_LENGTH:  1000
EPISODE:  1000 	REWARD:  -1000.0 	MEAN_REWARD:  -1000.0 	EPISODE_LENGTH:  1000


In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=mountaincar_noise_config),
    env_id = environments.MOUNTAINCAR_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: mountain_car_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -984.0074826301465 	MEAN_REWARD:  -998.95 	EPISODE_LENGTH:  1000
EPISODE:  200 	REWARD:  -1006.6954177370483 	MEAN_REWARD:  -999.26 	EPISODE_LENGTH:  1000
EPISODE:  300 	REWARD:  -999.4489016804749 	MEAN_REWARD:  -1000.41 	EPISODE_LENGTH:  1000
EPISODE:  400 	REWARD:  -1005.9196899050198 	MEAN_REWARD:  -1000.2 	EPISODE_LENGTH:  1000
EPISODE:  500 	REWARD:  -996.9460177617744 	MEAN_REWARD:  -1000.08 	EPISODE_LENGTH:  1000
EPISODE:  600 	REWARD:  -1005.6408451495033 	MEAN_REWARD:  -999.07 	EPISODE_LENGTH:  1000
EPISODE:  700 	REWARD:  -1002.8574841673754 	MEAN_REWARD:  -1000.17 	EPISODE_LENGTH:  1000
EPISODE:  800 	REWARD:  -1003.0486212221472 	MEAN_REWARD:  -999.14 	EPISODE_LENGTH:  1000
EPISODE:  900 	REWARD:  -1003.0506259195664 	MEAN_REWARD:  -1001.53 	EPISODE_LENGTH:  1000
EPISODE:  1000 	REWARD:  -992.5036966097842 	MEAN_REWARD:  -999.88 	EPISODE_LENGTH:  1000


# **Analysis & Result**

The following cells will show the score of the agent on each environment. The same scoring method will be used to evaluate your agent on a set of test environments.

In [ ]:
# *** PLEASE DONT EDIT THE CONTENTS OF THIS CELL ***
analyzer = Analyzer(os.environ.get('RESULTS_DIR'))
analyzer.print_scores()

╒════════════════════╤═══════════╕
│ ENVIRONMENT        │     SCORE │
╞════════════════════╪═══════════╡
│ catch              │ 0.000625  │
├────────────────────┼───────────┤
│ catch_noise        │ 0.0025    │
├────────────────────┼───────────┤
│ cartpole           │ 0.0196665 │
├────────────────────┼───────────┤
│ cartpole_noise     │ 0.0195645 │
├────────────────────┼───────────┤
│ mountain_car       │ 0.1       │
├────────────────────┼───────────┤
│ mountain_car_noise │ 0.1       │
╘════════════════════╧═══════════╛


In [ ]:
# If you want a object to get the scores
analyzer.get_scores()

{'cartpole': 0.019666499999999986,
 'cartpole_noise': 0.01956450000000001,
 'catch': 0.0006250000000000699,
 'catch_noise': 0.0025000000000000022,
 'mountain_car': 0.1,
 'mountain_car_noise': 0.1}

## What is the score function

The score function is developed by the BSuite team at Deepmind. It is open source and available at https://github.com/deepmind/bsuite

The score measures behavioral aspects of the agent only, and does not take into account internal state of the agent. For more details read Section 2 of the [BSuite paper](https://openreview.net/forum?id=rygf-kSYwH). In this case we use only the "Basic" aspect of the agent's scoring system.

**It is not necessary to understand the score in order to improve your agent's performance**

# **Backend Evaluation**

THIS CODE WILL EVALUATE THE AGENT USING THE SPECIFIED CONFIGS FOR THE CORRESPONDING ENVIRONMENTS. DO NOT EDIT THE CONTENTS OF THIS CELL.

In [ ]:
## Do not edit this cell
if (os.environ.get('BACKEND_EVALUATOR') is not None):
    
    import backend_evaluator

    runs = {
        'catch': (
            backend_evaluator.CATCH, 
            catch_config),
        'catch_noise': (
            backend_evaluator.CATCH_NOISE, 
            catch_noise_config),
        'cartpole': (
            backend_evaluator.CARTPOLE, 
            cartpole_config),
        'cartpole_noise': (
            backend_evaluator.CARTPOLE_NOISE, 
            cartpole_noise_config),
        'mountaincar': (
            backend_evaluator.MOUNTAINCAR, 
            mountaincar_config),
        'mountaincar_noise': (
            backend_evaluator.MOUNTAINCAR_NOISE, 
            mountaincar_noise_config)
    }

    for run_name, run in runs.items():
        env_ids, config = run
        for env_id in env_ids:
            runner = Runner(env_id=env_id,
                            agent=RLAgent(agent_config=config),
                            verbose=False,
                            eval=True)
            runner.play_episodes()

# Submit to AIcrowd 🚀

**NOTE: PLEASE SAVE THE NOTEBOOK BEFORE SUBMITTING IT (Ctrl + S)**

In [ ]:
# ! aicrowd notebook submit --no-verify -c iitm-rl-final-project -a assets

Mounting Google Drive 💾
Your Google Drive will be mounted to access the colab notebook
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g7GbK62SPaGCEmiIfhtRfMkduYY4ypqzLM9Q-kZ8huyOJQyO9se7gY
Mounted at /content/drive
Using notebook: /content/drive/MyDrive/Colab Notebooks/IITM_RL_Final_Project_Release.ipynb for submission...
Scrubbing API keys from 